In [ ]:
%sh cd .. && uv sync --active

UsageError: Line magic function `%sh` not found (But cell magic `%%sh` exists, did you mean that instead?).


In [ ]:
def widget(name, default):
    try:
        if value:= dbutils.widgets.get(name):
            return value
    except Exception:
        pass
    dbutils.widgets.text(name, default)
    return default

CATALOG_NAME = widget("CATALOG_NAME", "iot_dev")
SCHEMA_NAME  = widget("SCHEMA_NAME", "iot_ingest")
MODEL_ID     = widget("MODEL_ID", "yolo_object_detect")
ALIAS        = widget("ALIAS", "champion")
REGISTERED_MODEL_NAME = f"{CATALOG_NAME}.{SCHEMA_NAME}.{MODEL_ID}"
EXPERIMENT_PATH = f"/Shared/{REGISTERED_MODEL_NAME}"


NameError: name 'dbutils' is not defined

In [0]:
import mlflow.pyfunc
from typing import Any



class YoloDetectionModel(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        import os
        import tempfile
        from pathlib import Path

        yolo_config_dir = (
            Path(tempfile.gettempdir())
            / CATALOG_NAME
            / SCHEMA_NAME
            / MODEL_ID
            / "yolo_config"
        )
        yolo_config_dir.mkdir(parents=True, exist_ok=True)
        os.environ["YOLO_CONFIG_DIR"] = str(yolo_config_dir)

        from ultralytics import YOLO

        self.model = YOLO(context.artifacts["weights"])
        self.class_names = self.model.names  # id -> label

    def _decode(self, b64: str):
        import base64
        import numpy as np
        import cv2

        arr = np.frombuffer(base64.b64decode(b64), dtype=np.uint8)
        return cv2.imdecode(arr, cv2.IMREAD_COLOR)

    def predict(self, context, df):
        import pandas as pd

        images = [self._decode(b) for b in df["image_base64"]]
        results = self.model(images, conf=0.25, batch=len(images))

        rows: list[dict[str, Any]] = []
        for r in results:
            boxes = []
            if r.boxes is not None:
                xyxy = r.boxes.xyxy.cpu().numpy()
                conf = r.boxes.conf.cpu().numpy()
                cls = r.boxes.cls.cpu().numpy()

                for i in range(len(xyxy)):
                    class_id = int(cls[i])
                    boxes.append(
                        {
                            "label": self.class_names[class_id],
                            "confidence": float(conf[i]),
                            "x1": float(xyxy[i][0]),
                            "y1": float(xyxy[i][1]),
                            "x2": float(xyxy[i][2]),
                            "y2": float(xyxy[i][3]),
                        }
                    )

            rows.append({"boxes": boxes})

        return pd.DataFrame(rows)

In [0]:
from mlflow.models.signature import ModelSignature
from mlflow.types import Schema, ColSpec, DataType


import io
import base64
import pandas as pd
from PIL import Image
import mlflow


input_schema = Schema([ColSpec(DataType.string, "image_base64")])
output_schema = Schema([ColSpec(DataType.string, "json_result")])

signature = ModelSignature(
    inputs=input_schema,
    outputs=output_schema,
)

# create a tiny valid base64 image for input_example
buf = io.BytesIO()
Image.new("RGB", (2, 2), (255, 255, 255)).save(buf, format="PNG")
png_bytes = buf.getvalue()
image_base_64 = base64.b64encode(png_bytes).decode("utf-8")

# validate base64 image
Image.open(io.BytesIO(base64.b64decode(image_base_64))).verify()

input_example = pd.DataFrame({"image_base64": [image_base_64]})

mlflow.set_experiment(EXPERIMENT_PATH)

with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="model",
        python_model=YoloDetectionModel(),
        artifacts={
            "weights": "yolo11n.pt",
        },
        pip_requirements=["ultralytics"],
        input_example=input_example,
        signature=signature,
    )

model_uri = f"runs:/{run.info.run_id}/model"

In [0]:
from mlflow.tracking import MlflowClient

mv = mlflow.register_model(
    model_uri=model_uri,
    name=REGISTERED_MODEL_NAME,
)

MlflowClient().set_registered_model_alias(
    name=REGISTERED_MODEL_NAME,
    alias=ALIAS,
    version=mv.version,
)

print(mv.version)